In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
# !conda install -c conda-forge opencv
import sys
# !{sys.executable} -m pip install opencv-python
import cv2
# !{sys.executable} -m pip install tensorflow
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall, Precision

from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
def load_data(path,split = 0.1): # splitting the dataset into three categories as usual : 80:training, 10:validation, 10:testing
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    
    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    
    print("Total size: ",total_size)
    
    train_x, valid_x = train_test_split(images, test_size = valid_size, random_state = 42)
    train_y, valid_y = train_test_split(masks, test_size = valid_size, random_state = 42)
    
    train_x, test_x = train_test_split(train_x, test_size = test_size, random_state = 42)
    train_y, test_y = train_test_split(train_y, test_size = test_size, random_state = 42)
    
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y) 

In [4]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256, 256))
    x = x/255.0 # size is (256, 256, 3)
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256, 256))
    x= x/255.0
    x= np.expand_dims(x, axis =-1) #size is (256, 256, 1)
    x = x.astype(np.float32)
    return x

In [5]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape((256, 256, 3))
    y.set_shape((256, 256, 1))
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [6]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, (3, 3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

def build_model():
    size = 256
    num_filters = [32, 48, 64, 128]
    inputs = Input(shape = (size, size, 3))
    
    skip_x= []
    x = inputs
    
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2,2))(x)
        
    x = conv_block(x, num_filters[-1])
    num_filters.reverse()
    skip_x.reverse()
    
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)
        x = Dropout(0.2)(x)
        
    x = Conv2D(1, (1, 1), padding = "same")(x)
    x = Activation("sigmoid")(x)
    
    return Model(inputs, x)

In [7]:
model = build_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 256, 256, 32)      │             896 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256, 256, 32)      │             128 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 256, 256, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 256, 256, 32)      │           9,248 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 256, 256, 32)      │             128 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 256, 256, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 32)      │               0 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 128, 128, 48)      │          13,872 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 128, 128, 48)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 128, 128, 48)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 128, 128, 48)      │          20,784 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 128, 128, 48)      │             192 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 128, 128, 48)      │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 64, 64, 48)        │               0 │ activation_3[0][0]         │
│ (MaxPooling2D)                │                           │               

 Total params: 1,323,201 (5.05 MB)

 Trainable params: 1,320,513 (5.04 MB)

 Non-trainable params: 2,688 (10.50 KB)

In [8]:
def iou(y_true, y_pred):
  intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
  union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3]) - intersection
  iou = (intersection + 1e-15) / (union + 1e-15)
  return tf.reduce_mean(iou)
# def iou(y_true, y_pred):
#     def f(y_true, y_pred):
#         intersection = (y_true * y_pred).sum()
#         union = y_true.sum() + y_pred.sum() - intersection
#         x = (intersection + 1e-15)/(union + 1e-15)
#         x = x.astype(np.float32)
        
#         return x
#     return tf.numpy_function(f, [y_true, y_pred], tf.float32)
    # intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    # union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) - intersection
    # iou = (intersection + K.epsilon()) / (union + K.epsilon())
    # return iou

In [9]:
path = "local-dataset/"
(train_x, train_y),(valid_x, valid_y), (test_x, test_y) = load_data(path)
print("Train size: ",len(train_x),"/nValidation size: ",len(valid_x),"/nTest size: ",len(test_x))

Total size:  500
Train size:  400 /nValidation size:  50 /nTest size:  50


In [10]:
batch = 8
lr = 1e-4
epochs = 50

In [11]:
train_dataset = tf_dataset(train_x, train_y, batch = batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch = batch)

In [12]:
model = build_model()       
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", Recall(), Precision(), iou]
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = metrics)

In [13]:
callbacks = [
    ModelCheckpoint("files/model.keras"),
    ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3),
    CSVLogger("files/data.csv"),
    TensorBoard(),
    EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights= False)
]

In [14]:
print(valid_dataset, train_dataset)

<_RepeatDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))> <_RepeatDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>


In [15]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch
    
if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1
with tf.device('/gpu:0'):
    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 372s 7s/step - acc: 0.7750 - iou: 0.0913 - loss: 0.5574 - precision: 0.1790 - recall: 0.3416 - val_acc: 0.9016 - val_iou: 0.0848 - val_loss: 0.5807 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 327s 7s/step - acc: 0.8875 - iou: 0.1068 - loss: 0.3867 - precision: 0.3978 - recall: 0.2843 - val_acc: 0.9016 - val_iou: 0.0762 - val_loss: 0.4488 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 297s 6s/step - acc: 0.9067 - iou: 0.1256 - loss: 0.3341 - precision: 0.5306 - recall: 0.3916 - val_acc: 0.9016 - val_iou: 0.0651 - val_loss: 0.3838 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 297s 6s/step - acc: 0.9157 - iou: 0.1455 - loss: 0.3045 - precision: 0.5910 - recall: 0.4597 - val_acc: 0.9016 - val_iou: 0.0519 - val_loss: 0.3625 - val_precisi

In [41]:
def read_image2(path):
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256, 256))
    x= x/255.0 # size is (256, 256, 3)
    x = x.astype(np.float32)
    return x

def read_mask2(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256, 256))
    x = np.expand_dims(x, axis =-1) #size is (256, 256, 1)
    x = x.astype(np.float32)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask ,mask]
    mask = np.transpose (mask, (1, 2, 0))
    return mask

In [43]:
test_dataset = tf_dataset(test_x, test_y, batch = batch)
test_steps = len(test_x)//batch
if len(test_x) % batch != 0:
    test_steps += 1

with CustomObjectScope({'iou': iou}):
    model = tf.keras.models.load_model('files/model.h5')

In [45]:
#Seeding 
np.random.seed(42)
tf.random.set_seed(42)

ds = tf_dataset(test_x, test_y)
for x, y in ds:
    print(x.shape, y.shape)
    break

(8, 256, 256, 3) (8, 256, 256, 1)


In [47]:
model.evaluate(test_dataset, steps = test_steps)

7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - acc: 0.1073 - iou: 0.0896 - loss: 0.7044 - precision_5: 0.1069 - recall_5: 0.9993


[0.7037158012390137,
 0.11570892482995987,
 0.9991189241409302,
 0.1160530149936676,
 0.09475425630807877]

In [48]:
os.makedirs("results", exist_ok=True)
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total = len(test_x)):
    x = read_image2(x)
    y = read_mask2(y)
    y_pred = model.predict(np.expand_dims(x, axis = 0))[0]
    y_pred = y_pred > 0.5
    h, w, _ = x.shape

    white_line = np.ones((h, 10, 3)) * 255

    all_images=[
        x * 255.0,white_line,
        mask_parse(y),white_line,
        mask_parse(y_pred) * 255.0
        ]

    image = np.concatenate(all_images, axis = 1)
    cv2.imwrite(f"results/{i}.png", image)


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


  2%|█▋                                                                                 | 1/50 [00:02<02:10,  2.67s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


  4%|███▎                                                                               | 2/50 [00:03<01:27,  1.82s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [01:01<00:02,  1.23s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [01:03<00:01,  1.22s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:04<00:00,  1.29s/it]
